In [1]:
!python --version

In [2]:
import os
import sys
import datetime
import requests
import urllib.request
import json
import zipfile
import pandas as pd
import tweepy
import psycopg2
from sqlalchemy import create_engine
from datetime import date, timedelta

sys.path.append('../')
from envir import config

In [3]:
zip_url = 'https://data.ny.gov/api/views/juva-r6g2/rows.csv?accessType=DOWNLOAD'
urllib.request.urlretrieve(zip_url, config.shared+'ny_zip_demographics.csv')

('/gws_gpfs/projects/project-nyc_food_health/workspace/share/ny_zip_demographics.csv',
 <http.client.HTTPMessage at 0x7fec427d1fd0>)

In [4]:
zip_url = 'https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_zcta_national.zip'
urllib.request.urlretrieve(zip_url, config.shared+'2019_Gaz_zcta_national.zip')
    
with zipfile.ZipFile(config.shared+'Gaz_zcta_national_2019.zip', 'r') as zip_ref:
    zip_ref.extractall(config.shared)

In [5]:
us_zips = pd.read_csv(config.shared+'2019_Gaz_zcta_national.txt', sep="\t")
us_zips.head()

,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,601,166659747,799292,64.348,0.309,18.180555,-66.749961
1,602,79307534,4428428,30.621,1.710,18.361945,-67.175597
2,603,81887188,181412,31.617,0.070,18.455183,-67.119887
3,606,109579993,12487,42.309,0.005,18.158327,-66.932928
4,610,93013428,4172059,35.913,1.611,18.294032,-67.127156


In [6]:
ny_zips = pd.read_csv(config.shared+'ny_zip_demographics.csv', sep=',')
nyc_zips = ny_zips[ny_zips['County Name'].isin(['New York', 'Kings', 'Queens','Richmond','Bronx'])]
nyc_zips = pd.merge(nyc_zips, us_zips, how='inner', left_on='ZIP Code', right_on = 'GEOID')
nyc_zips.head()

,County Name,State FIPS,County Code,County FIPS,ZIP Code,File Date,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,Bronx,36,5,36005,10472,07/25/2007,10472,2729341,0,1.054,0.000,40.829556,-73.869310
1,Bronx,36,5,36005,10463,07/25/2007,10463,3755855,345779,1.450,0.134,40.880678,-73.906540
2,New York,36,61,36061,10463,07/25/2007,10463,3755855,345779,1.450,0.134,40.880678,-73.906540
3,Bronx,36,5,36005,10468,07/25/2007,10468,2773214,45100,1.071,0.017,40.868940,-73.899995
4,Bronx,36,5,36005,10457,07/25/2007,10457,2743123,0,1.059,0.000,40.847162,-73.898663


In [11]:
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
search_words=['doordash', 'ubereats', 'postmates', 'grubhub', 'seamless', 'caviar']
# query_list = map(lambda x:x+" -filter:retweets", search_words) # remove retweets
fr_date = date.today() - timedelta(7) # twitter api only allows upto 7 days on the basic service

In [23]:
for query in query_list:
    tweets = tweepy.Cursor(api.search, q=query, lang="en", since=fr_date).items(5)
    

In [125]:
tweets = tweepy.Cursor(api.search, q='order doordash -filter:retweets', geocode = '40.7128,-74.0060,30km',
                       lang="en", since=fr_date).items(5)

In [120]:
for t in tweets:
    print(t.text)

@DoorDash_Help So i get my order cancelled due to "technical difficulties" (understandable), but i can't have my mo… https://t.co/ejlzrwdj5Q
@DoorDash I hope I really did get credited for the dasher leaving my food order on the floor in the lobby of my bui… https://t.co/z6BUVZuzDE
@suatkilicMD @CBSNews And its a lot. If you do rhe same order on doordash, grub hub and ubereats, the final price i… https://t.co/4jGw2vigDL
@dangerousssdi No I had credit cause DoorDash fucked up my order before 😂
Welp @DoorDash, you fucked me over for the very LAST TIME! Thank you for delivering my order to the wrong address!… https://t.co/SM30dlsOgA


In [126]:
for t in tweets2:
    print (t.created_at,t.text,t.place, t.coordinates)

2020-12-02 17:59:18 @DoorDash_Help So i get my order cancelled due to "technical difficulties" (understandable), but i can't have my mo… https://t.co/ejlzrwdj5Q None None
2020-12-02 17:54:11 @DoorDash I hope I really did get credited for the dasher leaving my food order on the floor in the lobby of my bui… https://t.co/z6BUVZuzDE None None
2020-12-02 15:13:15 @suatkilicMD @CBSNews And its a lot. If you do rhe same order on doordash, grub hub and ubereats, the final price i… https://t.co/4jGw2vigDL None None
2020-12-02 07:31:12 @dangerousssdi No I had credit cause DoorDash fucked up my order before 😂 None None
2020-12-02 01:18:44 Welp @DoorDash, you fucked me over for the very LAST TIME! Thank you for delivering my order to the wrong address!… https://t.co/SM30dlsOgA None None


In [128]:
for t in tweets2:
    print (t.created_at,t.text,t.place, t.coordinates)

TypeError: 'method' object is not iterable

In [127]:
tweets2 = tweets.next

In [35]:
place = api.geo_search(query="New York City", granularity="city")
place_id = place[0].id

In [36]:
place_id

'7b02fbddf4d9f2c6'

In [80]:
for tweet in tweepy.Cursor(api.search,
                           q="doordash -filter:retweets",
                           geocode="40.7128,-74.0060,30km",
                           #result_type="recent",
                           include_entities=True,
                           lang="en").items():
    print (tweet.created_at,tweet.text,tweet.place, tweet.coordinates)

2020-12-02 17:19:54 @DoorDash shoutout for the 835?minute wait, still don’t have my food https://t.co/I1tTKkCdYI None None
2020-12-02 16:58:20 Super hyped for next weeks IPOs😁I waited ALL YEAR for this!🙌🏽👏🏽
#stocks #IPO #stockstowatch #airbnb #doordash… https://t.co/l1hrR4sIN9 None None
2020-12-02 16:16:42 @Uber_Support do not you have already done so and incorrectly made a poor decision. I am already working with DoorD… https://t.co/O1oIDSClbY None None
2020-12-02 16:15:02 @Lightshow_Leemz Lemme see if they on DoorDash None None
2020-12-02 16:10:49 $DASH - DoorDash attracts a second bull as IPO approaches https://t.co/7tPQSiaA9P None None
2020-12-02 15:55:34 @locduplilmama I only saw it on doordash so idk what it even looks like. all I saw was seafood lmaoo None None
2020-12-02 15:51:25 can’t wait for everyone’s Doordash Wrapped Place(_api=<tweepy.api.API object at 0x7f5d1f868c90>, id='11db9e913ad41000', url='https://api.twitter.com/1.1/geo/id/11db9e913ad41000.json', place_type='poi',

2020-12-01 16:45:08 Food delivery company DoorDash set a price range on an estimated $2.6 billion initial public offering.… https://t.co/qa5ydtCVlH None None
2020-12-01 16:44:40 I’ve never used a postmates kind of service because DoorDash unlawfully steals tips. 

If you deliver shit and don’… https://t.co/xR6wyTF3Da None None
2020-12-01 16:21:09 @KJ_IRL @DoorDash @steak_umm For some reason, Twitter is not showing the total votes at the moment, but more people are indeed weighing in! None None
2020-12-01 16:09:43 @DearDara Anticompetitive monopoly? For it to be a monopoly, it would need 75-100% market share, whereas the pro fo… https://t.co/VfIg1dopZY None None
2020-12-01 15:57:21 OPTION 1: @DoorDash stepped up quickly after the pandemic hit, launching its #OpenForDelivery campaign in mid-to-la… https://t.co/41cat4z8cU None None
2020-12-01 15:57:21 This is the 8th matchup in Round 1 of our #AdOfTheYear Reader's Choice bracket! 🏆

Which do you think was the more… https://t.co/z0qqRph3Zb

2020-11-30 21:12:27 @obidoess @DoorDash_Help @DoorDash LMFAOO None None
2020-11-30 20:53:14 .@DoorDash aims to double its valuation in #IPO to $32B from $16B less than 6 months ago. Underscores demand for me… https://t.co/GFxhH1v85H None None
2020-11-30 20:30:00 Food Delivery Apps Are Booming. Their Workers Are Often Struggling.

Delivery drivers have been essential to feedin… https://t.co/4Jd1EaYynI None None
2020-11-30 20:17:36 A fresh batch of our Jumbo BBQ Wings is ready for Monday Night Football! Order yours tonight at… https://t.co/XNy4ejvTqT Place(_api=<tweepy.api.API object at 0x7f5d1f868c90>, id='011add077f4d2da3', url='https://api.twitter.com/1.1/geo/id/011add077f4d2da3.json', place_type='city', name='Brooklyn', full_name='Brooklyn, NY', country_code='US', country='United States', contained_within=[], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x7f5d1f868c90>, type='Polygon', coordinates=[[[-74.041878, 40.570842], [-73.855673, 40.570842], [-73.855673, 40.739434],

2020-11-30 13:27:12 DoorDash seeks valuation of up to $30 billion in IPO https://t.co/mpnU2kseAS None None
2020-11-30 13:25:12 I was going to run to Starbucks, until I saw that rain chileee......DoorDash it is None None
2020-11-30 13:24:08 AirBnB, DoorDash Raise Share Price Targets As US IPO Market Booms https://t.co/0xYt02W15s https://t.co/FeTiYkNnVg None None
2020-11-30 13:22:29 GOOD MORNING!
Only 51 Days until Inauguration Day

Here’s your @cheddar business headlines for Nov 30

- Airbnb, Do… https://t.co/SQDi9mhUl5 None None
2020-11-30 13:21:49 U.K. to become the first country to roll out a Covid-19 vaccine, Moderna requests emergency authorizations for its… https://t.co/JiVeRM0z57 None None
2020-11-30 12:57:41 Airbnb and DoorDash Are Said to Target Higher IPO Valuations - Bloomberg https://t.co/sjCUPIVovE None None
2020-11-30 12:43:41 It's not just that Uber Eats, DoorDash &amp; GrubHub super-exploit delivery workers, they also ripoff restaurants by ch… https://t.co/uvkqt1o28v Non

2020-11-28 23:31:17 One thing about me imma call doordash complain &amp; get all my money back.. partial credit my ass SMD None None
2020-11-28 22:55:12 @DoorDash is Trash!!! Their customer service is horrible and their drivers are even worse! None None
2020-11-28 22:17:52 @NovemberWinning @AndreaT314 @HankScorpio325 @DoorDash And are you speaking of me? None None
2020-11-28 22:01:22 $anai000 someone pay for my doordash 😔 None None
2020-11-28 21:59:01 the fucking Doordash fees are disrespectful None None
2020-11-28 21:23:08 *sighs harder*
WHY IS DOORDASH SO EXPENSIVE None None
2020-11-28 21:02:06 I am going to delete DoorDash. This is getting out of hand. None None
2020-11-28 20:09:38 doordash pass is actually worth it son. that delivery fee be murderin u None None
2020-11-28 20:05:14 I saw two people in Camry pull up to my house with vest on dropping off my package like it was a DoorDash delivery.....Amazon ion like that! None None
2020-11-28 17:19:34 DoorDash so damn annoying, I orde

2020-11-26 02:00:14 Candle lit playing spider man waiting for doordash None None
2020-11-26 01:14:47 @hacks4pancakes @AccidentalCISO Doordash and Dairy Queen definitely getting my money tonight None None
2020-11-26 00:28:54 @DoorDash is the app down again None None
2020-11-25 23:44:30 @DoorDash_Help They need to stop lying on the app with these unrealistic wait times for customers. Its restaurant i… https://t.co/mR7KOuBjlZ None None
2020-11-25 23:43:57 Lmaoooo my dumbass would click on that Instagram spam text so someone could hack my doordash account and order $250… https://t.co/MkviXF0ava None None
2020-11-25 23:39:44 @DoorDash_Help They rate dashers and that completely sucks because customers don't rate you as driver they will you… https://t.co/BUTSM337wF None None
2020-11-25 23:39:30 @benegotherit had a doordash order at 3am the other day for a pickle and a big gulp from 7-11. I drove 15 minutes t… https://t.co/jrhTjmj5sE None None
2020-11-25 23:36:18 @JoeBiden They are supporting 

2020-11-25 16:59:03 I DoorDash, Postmates, and UberEats like it’s online shopping https://t.co/VVyyqHof67 None None
2020-11-25 16:55:32 This is great news &amp; highlights need for the permanent 3rd-party delivery reform that @MarkGjonajNY has fought for.… https://t.co/ih0GpFu82e None None
2020-11-25 16:54:00 @DoorDash please put back the restaurant addresses on delivery orders. I need to know where my food is coming from.… https://t.co/PX4IP5f8QK None None
2020-11-25 16:45:09 Paypal Digital Gifts: 10.5% discount on DoorDash #PaypalDigitalGifts #DoorDash https://t.co/IRgFo7FQ4Q None None
2020-11-25 16:43:28 @SC_Griffith @DoorDash @AGKarlRacine How so? What was the new system to hide it, from what you heard? None None
2020-11-25 16:27:11 Cancel @DoorDash https://t.co/UNjr4UHIDH Place(_api=<tweepy.api.API object at 0x7f5d1f868c90>, id='011add077f4d2da3', url='https://api.twitter.com/1.1/geo/id/011add077f4d2da3.json', place_type='city', name='Brooklyn', full_name='Brooklyn, NY', country_c

2020-11-24 07:33:11 @SoraTBE @DoorDash is TRASH. I outright refuse to use them. None None
2020-11-24 06:07:04 Just redownloaded the doordash app hello employed twitter None None
2020-11-24 05:22:58 Doordash is so ghetto, I miss Uber. Unblock me please ❤️ @UberEats None None
2020-11-24 03:40:27 Only two times I ordered door dash and I’m 2/2 with DoorDash drivers not delivering me my food, like bro??😂😂 None None
2020-11-24 03:22:50 Airbnb, DoorDash and Affirm: IPOs' Year-End Rush - Bloomberg https://t.co/UTMFHlpZZ5 None None
2020-11-24 02:47:03 9:23 go to doordash for diamonds see it’s closed don’t believe it it can’t be.... call their line is busy maybe squ… https://t.co/DXkpKvj9EJ None None
2020-11-24 02:42:29 Y’all ever get so zooted that you don’t forget your mask but you forget your pants when you open the door for your @DoorDash order None None
2020-11-24 02:19:58 BREAKING: 
Thrilled to be selected by the prestigious @IRGRealGrowth as one of the 'IRG100' – for its IRG100 Leader… ht

In [85]:
# LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the 
 # geographic area from which you will stream tweets. The first two define the southwest 
 # corner of the box and the second two define the northeast corner of the box. 
LOCATIONS = [-74.260711, 40.495003, -66.947028, 49.384472] 

stream_listener = MyStreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
stream = tweepy.Stream(auth=, listener=stream_listener) 
stream.filter(locations=LOCATIONS)

NameError: name 'auth1' is not defined

In [84]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        print(status.text)

In [86]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [90]:
LOCATIONS = [-74.260711, 40.495003, -66.947028, 49.384472] 
myStream.filter(track=['doordash'],locations=LOCATIONS)

This #HR job might be a great fit for you. Click the link in our bio to see it and more. Coordinator-HR Campus (Ful… https://t.co/a65vYBxs4U
not to get tOo PoLiTiCaL er nuthin but the fact the proud boy dweebs ruined the 👌 emoji aka “TOOO SWEEEET might jus… https://t.co/BxPIGXseD9
@MayorSPH 30 min entro of the individual now my game plan 1hr questions outside the mayors office interview connect… https://t.co/3RSMLsQn7l
Finna go get braces again.
Happy Birthday Gary 🎁🎁🎁⚾️ https://t.co/BD81FosMtN
Listen to the range and precision!
@SNUKgaming I'm more button masher and joystick jiggler https://t.co/sha8Xdz2OL
@AlyssaRaeTaglia @RyanKristafer Also cross fit.  How do u k is someone does CrossFit? They’ll tell u.  😏
@OnlyInBOS what’s this gotta do with Boston?
Hella grateful for the overflow of creative direction bookings. I appreciate everyone for not only referring me but… https://t.co/owcfEo97Yg
@anthony67116458 You know makes me happy?
#CountEveryLegalVote #FightBack #FightLikeAFlynn #St

KeyboardInterrupt: 

In [91]:
myStream.disconnect()

In [129]:
tweepy.Cursor?